In [118]:
import json
import os
import geojson

# Preparation

In [119]:
max_x_tile = 40 
max_y_tile = 40 

# Format should always be "XXXX"
all_tiles = ["{:04d}".format(number) for number in range(max_x_tile*max_y_tile)]

#print(os.getcwd())
#print(os.listdir('40_40/GeoJSON'))

# Functions

In [120]:
def find_site_types(sitetype: str) -> int:
    '''
    Given a json, find the int_sitetype_limes (now int_sitety) and output a classifer as either 0, 1, or 2
    0: Bombentrichter
    1: Grabhügel
    2: Meilerpodium
    Input: str
    Output: int
    '''
    if sitetype == 'Bombentrichter':
        return 0
    elif sitetype == 'Grabhügel':
        return 1
    elif sitetype == 'Meilerpodium':
        return 2
    else:
        raise Exception("Did not find Bombentrichter, Grabhügel, or Meilerpodium")

In [121]:
def gen_x_y_points(feature: list):
    '''
    Takes a feature and returns a list of the x and y coordinates aswell as all points
    Input: list
    Output: list, list, and list
    '''
    points = feature['geometry']['coordinates'][0] 
    x = [p[0] for p in points]
    y = [p[1] for p in points]
    return x, y, points

In [122]:
def calc_width(x: list) -> float:
    '''
    Given a list of all x values, identifies the width of the feature
    Input: list
    Output: float
    '''
    return max(x) - min(x)

In [123]:
def calc_height(y: list) -> float:
    '''
    Given a list of all y values, identifies the height of the feature
    Input: list
    Output: float
    '''
    return max(y) - min(y)

In [124]:
def find_parameters(feature: list) -> list:
    '''
    Takes a single feature, extracts the necessary information, and returns a list of paramters.
    The parameters are in the specified order:
    [class, x_center, y_center, width, height]
    Input: list
    Output: list
    '''
    parameters = []
    
    x, y, points = gen_x_y_points(feature)
    x_cen, y_cen = ( sum(x) / len(points), sum(y) / len(points))
    
    norm_x, norm_y = normalize_parameters(x_cen, y_cen)
    
    parameters.append(find_site_types(feature['properties']['int_sitety']))
    parameters.append(norm_x)
    parameters.append(norm_y)
    parameters.append(calc_width(x))
    parameters.append(calc_height(y))
    return parameters

In [127]:
def normalize_parameters(x: float, y: float):
    '''
    Takes x and y value and normalizes them
    Origin (0,0) is the lower left corner
    Input: float
    Output: float
    '''
    global_min_x = 314084.5
    global_min_y = 5585151.5 + 30

    image_width = 1211
    image_height = 907
    
    # 1 in the grid is equal to 1 pixel
    x = x - global_min_x
    y = global_min_y - y
    
    x = x%image_width
    y = y%image_height
    
    y = 907 - y
    
    norm_x = x/image_width
    norm_y = y/image_height
    
    #print(x, y, norm_x, norm_y)
    
    return norm_x, norm_y
    
    
#normalize_parameters(319390.35907340754, 5578030.177083888) #192

# Get Parameters

In [128]:
print("Start extracting the parameters from the feature files")

base_src_path = "GeoJSON"
base_target_path = "dataset/labels"

for i in range(max_x_tile*max_y_tile):
    
    file_name_geo = all_tiles[i] + ".geojson"
    src_path = os.path.join(base_src_path, file_name_geo)
    #print(src_path)
    
    # Define target location e.g.'40_40/GeoJSON/0001.txt'
    file_name_txt = all_tiles[i] + ".txt"
    target_path = os.path.join(base_target_path, file_name_txt)
    #print(target_path)
    
    # Load all GeoJSON files that exist
    try:
        f = open(src_path)
        json_file = geojson.load(f)
        print(file_name_geo + "File found")
    
    except:
        print(file_name_geo + " does not exist")
        
        
    #print(src_path)
    
    all_my_parameters = []

    # Get all needed parameters
    #try:
    for j in range(len(json_file["features"])):
            #print("in")
        
        all_my_parameters.append(find_parameters(json_file['features'][j]))   # Add for each feature the parameters that YOLOv5 needs
    
    
            #print("H" + type(all_my_paraneters))
    #except:
        #print("Nothing detected in this image: " + file_name_geo)    
              
      
    # Test to see what features are in one image
    for j in range(len(all_my_parameters)):
        print(all_my_parameters[j])
    
    # Save the features to a TXT file seperates by lines
    with open(target_path, 'w') as target_file:
        target_file.write('\n'.join(([' '.join(map(str,j)) for j in all_my_parameters])))
    
print("Done")

Start extracting the parameters from the feature files
0000.geojson does not exist
[2, 0.6444994322020249, 0.30969415921635735, 22.745170999958646, 22.746027999557555]
[2, 0.5951503103087629, 0.9758293225523392, 23.321295999980066, 23.321297000162303]
[2, 0.6624769369701868, 0.9921060631291935, 20.085671999957412, 8.117322999984026]
[2, 0.6753702954549502, 0.84522076976273, 15.939741000009235, 15.94034199975431]
[2, 0.42455845119877283, 0.19439013104766706, 23.62192700000014, 23.621927000582218]
[2, 0.3534822140548849, 0.023503379803869663, 20.273272999969777, 20.274035999551415]
0001.geojsonFile found
0002.geojsonFile found
0003.geojsonFile found
0004.geojsonFile found
0005.geojsonFile found
0006.geojsonFile found
0007.geojsonFile found
0008.geojsonFile found
0009.geojsonFile found
0010.geojsonFile found
0011.geojsonFile found
0012.geojsonFile found
0013.geojsonFile found
0014.geojsonFile found
0015.geojsonFile found
0016.geojsonFile found
0017.geojsonFile found
0018.geojsonFile found